# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/modulea function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1. with 

In [43]:
!conda create -n msds python=3.10.14 -y
!conda activate msds
!pip install --upgrade pycaret

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Asritha\anaconda3\envs\msds

  added / updated specs:
    - python=3.10.14


The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/win-64::bzip2-1.0.8-h2bbff1b_6 
  ca-certificates    pkgs/main/win-64::ca-certificates-2024.7.2-haa95532_0 
  libffi             pkgs/main/win-64::libffi-3.4.4-hd77b12b_1 
  openssl            pkgs/main/win-64::openssl-3.0.15-h827c3e9_0 
  pip                pkgs/main/win-64::pip-24.2-py310haa95532_0 
  python             pkgs/main/win-64::python-3.10.14-he1021f5_1 
  setuptools         pkgs/main/win-64::setuptools-75.1.0-py310haa95532_0 
  sqlite             pkgs/main/win-64::sqlite-3.45.3-h2bbff1b_0 
  tk                 pkgs/main/win-64::tk-8.6.14-h0416ee5_0 
  tzdata             pkgs/main/noarch::tzdata-2024a-h04d1e81_0 
  vc                 pkgs/main/win-64::vc-14.40-h2eaa2aa_1 
  vs2015_runtime   

In [1]:
import pandas as pd 
df=pd.read_csv('/Users/mallikarjunreddy/Downloads/prepared_churn_data.csv',index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
customerID,,,,,,,,
7590-VHVEG,1.0,0,0,3,29.85,29.85,0,29.850000
5575-GNVDE,34.0,1,1,2,56.95,1889.50,0,55.573529
3668-QPYBK,2.0,1,0,2,53.85,108.15,1,54.075000
7795-CFOCW,45.0,0,1,1,42.30,1840.75,0,40.905556
9237-HQITU,2.0,1,0,3,70.70,151.65,1,75.825000
...,...,...,...,...,...,...,...,...
6840-RESVB,24.0,1,1,2,84.80,1990.50,0,82.937500
2234-XADUH,72.0,1,1,0,103.20,7362.90,0,102.262500
4801-JZAZL,11.0,0,0,3,29.60,346.45,0,31.495455


In [2]:
# Display basic information about the dataset
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tenure             7043 non-null   float64
 1   PhoneService       7043 non-null   int64  
 2   Contract           7043 non-null   int64  
 3   PaymentMethod      7043 non-null   int64  
 4   MonthlyCharges     7043 non-null   float64
 5   TotalCharges       7043 non-null   float64
 6   Churn              7043 non-null   int64  
 7   charge_per_tenure  7043 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 495.2+ KB


(None,
             tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
 customerID                                                                  
 7590-VHVEG     1.0             0         0              3           29.85   
 5575-GNVDE    34.0             1         1              2           56.95   
 3668-QPYBK     2.0             1         0              2           53.85   
 7795-CFOCW    45.0             0         1              1           42.30   
 9237-HQITU     2.0             1         0              3           70.70   
 
             TotalCharges  Churn  charge_per_tenure  
 customerID                                          
 7590-VHVEG         29.85      0          29.850000  
 5575-GNVDE       1889.50      0          55.573529  
 3668-QPYBK        108.15      1          54.075000  
 7795-CFOCW       1840.75      0          40.905556  
 9237-HQITU        151.65      1          75.825000  )

In [3]:
pip install pycaret

Using legacy 'setup.py install' for pyod, since package 'wheel' is not installed.
    Running setup.py install for pyod: started
    Running setup.py install for pyod: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Asritha\asritha\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [8]:
automl = setup(df, target='Churn')

,Description,Value
0,Session id,1186
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7032, 9)"
4,Transformed data shape,"(7032, 9)"
5,Transformed train set shape,"(4922, 9)"
6,Transformed test set shape,"(2110, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [9]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7920,0.8361,0.5245,0.6342,0.5727,0.4370,0.4415,1.1040
gbc,Gradient Boosting Classifier,0.7918,0.8422,0.4870,0.6440,0.5539,0.4216,0.4290,1.1140
ada,Ada Boost Classifier,0.7905,0.8341,0.5031,0.6368,0.5606,0.4257,0.4318,0.3750
lda,Linear Discriminant Analysis,0.7873,0.8248,0.5039,0.6283,0.5578,0.4201,0.4255,0.0420
ridge,Ridge Classifier,0.7871,0.8249,0.4496,0.6473,0.5289,0.3972,0.4092,0.0390
lightgbm,Light Gradient Boosting Machine,0.7818,0.8282,0.5199,0.6060,0.5586,0.4150,0.4178,0.6530
rf,Random Forest Classifier,0.7737,0.8130,0.4756,0.5949,0.5270,0.3810,0.3860,0.9060
et,Extra Trees Classifier,0.7674,0.7939,0.4970,0.5730,0.5312,0.3778,0.3802,0.5910
knn,K Neighbors Classifier,0.7566,0.7524,0.4336,0.5540,0.4847,0.3291,0.3341,0.6660
svm,SVM - Linear Kernel,0.7391,0.7179,0.4484,0.5912,0.4678,0.3152,0.3464,0.0670


In [10]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1186, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
df.iloc[-2:-1].shape

(1, 9)

In [12]:
predict_model(best_model, df.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,tenure_monthly_ratio,tenure_total_ratio,Churn,prediction_label,prediction_score
customerID,,,,,,,,,,,
8361-LTMKD,4,1,0,3,74.400002,306.600006,0.053763,0.013046,1,1,0.6094


In [13]:
save_model(best_model, 'lr')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges',
                                              'tenure_monthly_ratio',
                                              'tenure_total_ratio'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=na...
                                                               fill_value=None,
            

In [14]:
import pickle

with open('lr_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [15]:
with open('lr_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [16]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1], dtype=int8)

In [20]:
loaded_lr = load_model('lr')

Transformation Pipeline and Model Successfully Loaded


In [21]:
predict_model(loaded_lr, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,tenure_monthly_ratio,tenure_total_ratio,prediction_label,prediction_score
customerID,,,,,,,,,,
8361-LTMKD,4,1,0,3,74.400002,306.600006,0.053763,0.013046,1,0.6094


In [63]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv('prepped_churn_data.csv', index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('lr')
    predictions = predict_model(model, data=df)

    # Check the column names
    print(predictions.columns)
    
    # Rename 'prediction_label' to 'Churn_prediction' if it exists
    if 'prediction_label' in predictions.columns:
        predictions.rename(columns={'prediction_label': 'Churn_prediction'}, inplace=True)
        
        # Replace values in the new column
        predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'}, inplace=True)
        
        return predictions['Churn_prediction']
    else:
        raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [64]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7931,0.8335,0.5249,0.6337,0.5742,0.4391,0.4425


Index(['tenure', 'PhoneService', 'Contract', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'tenure_monthly_ratio', 'tenure_total_ratio', 'Churn',
       'prediction_label', 'prediction_score'],
      dtype='object')
predictions:
customerID
7590-VHVEG       Churn
5575-GNVDE    No Churn
3668-QPYBK    No Churn
7795-CFOCW    No Churn
9237-HQITU       Churn
                ...   
6840-RESVB    No Churn
2234-XADUH    No Churn
4801-JZAZL    No Churn
8361-LTMKD       Churn
3186-AJIEK    No Churn
Name: Churn_prediction, Length: 7032, dtype: object


Summary

A Conda environment was set up with Python 3.10.14 and PyCaret installed to work with churn data. The data was loaded into a Pandas DataFrame and prepared for analysis, with 'Churn' as the target variable. The data was split into training and testing sets, and missing values were handled. After testing several algorithms, Logistic Regression was found to be the best, achieving an accuracy of 79.20% and an AUC score of 0.8361. The model was saved using both PyCaret and Python’s pickle module. It was then used to predict whether a specific customer was likely to churn, showcasing how effective PyCaret is for automating churn prediction.
